# 1. Noise-free data

In [ ]:
def simulate_sir_simple(lam, mu, I_0, T):
    """
    simulates the epidemy using the Euler forward scheme up to day T.
    lam = infection rate
    mu = recovery rate
    I_0 = initial percentage of infected people
    T = days to simulate
    """
    # Initialize variables
    S = 1.0 - I_0
    I = I_0
    R = 0.0
    
    # Lists to store observations and ODE variables
    X = []
    C = []
    
    for day in range(1, T+1):
        
        # Calculate new values using Euler forward scheme
        dS = -lam * S * I
        dI = lam * S * I - mu * I
        dR = mu * I

        S += dS
        I += dI
        R += dR

        # Append current values to the lists
        X.append((dS,dR))
        C.append((S, I, R))

    
    return observations, ode_variables


In [ ]:
# Example usage:
lambda_val = 0.2  # Infection rate
mu_val = 0.1     # Recovery rate
initial_infected = 0.01  # Initial number of infected people
num_days = 100   # Total number of days to simulate

observations, ode_variables = simulate_sir_simple(lambda_val, mu_val, initial_infected, num_days)

# You can then inspect the lists 'observations' and 'ode_variables' for the results.

# 2. Noisy data

# 3. Model misspecification detection

# 4. Sensitivity analysis